In [ ]:
import os
os.chdir('C:/Users/theodore.wong/cif/cities-cif')
import geemap, json
import pandas as pd
import geopandas as gpd
import ee
# ee.Authenticate()
ee.Initialize()
from city_metrix.layers import Layer, LandSurfaceTemperature, EsaWorldCover, EsaWorldCoverClass

FILEPATH = 'C:/Users/theodore.wong/danida_africa'

In [ ]:
boundary_filenames = os.listdir('{0}/boundaries'.format(FILEPATH))
boundary_filenames

In [ ]:
def percent_builtup_in_high_LST(gdf, matches):
    results = [None] * len(gdf)
    dones = []
    builtup_mask = EsaWorldCover(land_cover_class=EsaWorldCoverClass.BUILT_UP, year=2021)
    mean_LSTs_from_groupby = LandSurfaceTemperature().mask(builtup_mask).groupby(gdf['geometry']).mean()
    
    mean_LSTs = [None] * len(gdf)
    idx = 0
    for mean_LST in mean_LSTs_from_groupby:
        while mean_LSTs[idx] is not None:
            idx += 1
        mean_LSTs[idx] = mean_LST
        for match in matches[idx]:
            mean_LSTs[match] = mean_LST
        idx += 1
    
    for idx in range(len(gdf)):
        if not idx in dones:
            the_geom = gpd.GeoDataFrame(gdf.iloc[idx]).transpose().set_geometry('geometry').set_crs('epsg:4326').total_bounds
            bu = EsaWorldCover(land_cover_class=EsaWorldCoverClass.BUILT_UP, year=2021).get_data(the_geom) / 50
            lst = LandSurfaceTemperature().get_data(the_geom) > (mean_LSTs[idx] + 3)
            combo = bu * lst

            results[idx] = 100 * (float(combo.sum()) / float(bu.sum()))
            dones.append(idx)
            for i in matches[idx]:
                results[i] = results[idx]
                dones.append(i)
    return pd.Series(results).round(decimals=1)


In [ ]:
for fname in boundary_filenames:
    if fname.split('-')[3].split('.')[0][-1] != 'n':
        cityname = '{0} {1}'.format(fname.split('-')[1], fname.split('-')[2])
        with open('{0}/boundaries/{1}'.format(FILEPATH, fname), 'r') as ifile:
            json_obj = json.loads(ifile.read())
            gdf = gpd.GeoDataFrame.from_features(json_obj)
            matches = {rownum: [] for rownum in range(len(gdf))}
            for r_out in range(len(gdf)):
                for r_in in range(len(gdf)):
                    if not r_out == r_in:
                        if gdf.iloc[r_out]['geometry'] == gdf.iloc[r_in]['geometry']:
                            matches[r_out].append(r_in)
            gdf['percent_builtup_in_highLST'] = percent_builtup_in_high_LST(gdf, matches)
            with open('{0}/percent_builtup_in_highLST-{1}.geojson'.format(FILEPATH, cityname.replace(' ', '-')), 'w') as ofile:
                ofile.write(gdf.to_json())
            print(cityname)